# Affine Cipher

Affine cipher is a cipher technique to encrypt plain text using a 2 part key.
Assume we have the key pair (k1, k2) we can encrypt a plain text say P, we calculate the cipher text C as -

C = (P x k1 + k2) mod 26 (assuming we are limited to 26 character alphabet)

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters)

In [64]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    return result

Lets declare a plain text that we would need to encrypt.

In [65]:
P = 'Enemy is going to attack today. Please beware!'

Encoding this string, we get -

In [66]:
T = encode(P)
print(T)

ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [67]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key pair (k1, k2) that we would be using to encrypt the plain text.

In [68]:
from random import choice
k1_domain = [int(x) for x in range(26) if gcd(x, 26) == 1]
k2_domain = [int(x) for x in range(26)]
key = k1, k2 = (Zp(choice(k1_domain)), Zp(choice(k2_domain)))
print('Key -', key)

Key - (25, 24)


Using the definition of affine cypher, we can encrypt each character of text using -

C = (P x k1 + k2) mod 26

In [69]:
def affinecipher(text, cipher_key):
    cipher = ''
    k1, k2 = cipher_key
    for letter in text:
        text_value = (ord(letter) - ord('A')) % 26 # convert letter to its numerical value
        
        cipher_value = (text_value * k1 + k2) % 26 # convert to cipher value
        
        cipher += chr(cipher_value + ord('A')) # conver cipher value to character
        
    return cipher

def inv(x, m):
    x = x % m
    for i in range(m):
        if (x * i) % m == 1:
            return i
    return 1

def affinedecipher(cipher_text, cipher_key):
    text = ''
    k1, k2 = cipher_key
    for letter in cipher_text:
        cipher_value = (ord(letter) - ord('A')) % 26 # convert letter to its numerical value
        
        text_value = ((cipher_value - k2) * inv(k1, 26)) % 26 # convert to cipher value
        
        text += chr(int(text_value) + ord('A')) # conver cipher value to character
        
    return text

In [70]:
T = encode(P)
C = affinecipher(T, key)
D = affinedecipher(C, key)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key - {key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')

Given text - "Enemy is going to attack today. Please beware!"
Encoded - ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE
Key - (25, 24)
 

Decipher text - ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE


## Test Against Builtin Cipher

Now, we can test the result against the built in Affine Cipher in sagemath.

In [71]:
A = AffineCryptosystem(AlphabeticStrings())
E = A.encoding(P)
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key - ({k1}, {k2})')
C_test = A.enciphering(int(k1), int(k2), E)
D_test = A.deciphering(int(k1), int(k2), C_test)

# convert to python string
#C_test = str(C_test)
D_test = str(D_test)
print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')

Text - Enemy is going to attack today. Please beware!
Encoded - ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE
Key - (25, 24)
Cipher text - ULUMAQGSKQLSFKYFFYWOFKVYAJNUYGUXUCYHU
Decipher text - ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE


Comparing the built in cipher result with our implementation -

In [72]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

 
 	 ULUMAQGSKQLSFKYFFYWOFKVYAJNUYGUXUCYHU
Decipher Text 	 ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE 	 ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE

Implementatiokn is INCORRECT


## Cryptoanalysis

Since we know the key domain for the given cipher algorithm, we can run a brute force attack on the cryptosystem using a list of keys possible within the domain.

In [73]:
key_list = []
for i in k1_domain:
    for j in k2_domain:
        t = (i, j)
        key_list.append((i, j))
print(f'Total keys - {len(key_list)}')

Total keys - 312


We must now get a list of english words that can be used to detect existence of english words in our bruteforced decipher text.

A good list of 3000 most used english words is here -
https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt

We download the list of words and convert it to a list

In [74]:
import requests
url = 'https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt'

words_file = requests.get(url, allow_redirects=True)
words_file_obj = open('words.txt', 'wb')
words_file_obj.write(words_file.content)
words_file_obj.close()

In [75]:
words = open('words.txt').read().split()

In [76]:
words = [word.upper() for word in words]
print(f'Number of words in dictionary - {len(words)}')

Number of words in dictionary - 3000


In [77]:
# function to find english words in a string according to word list
def find_words(string):
    l = len(string)
    found = []
    for i in range(l):
        for j in range(i, l):
            word = string[i:j+1]
            if len(word) <= 1:
                continue
            if word in words:
                found.append(string[i:j+1])
    return found

Now, we must try to decipher the encrypted text using the list of keys we have and try to compare and count any english words found in the text. More the words detected, more likely is it that the key is correct.

In [78]:
import re
results = []

keys = {}
max_words = 0
for candidate in key_list:
    candidate_text = affinedecipher(C, candidate)
    found = find_words(candidate_text)
    if len(found) > 3:
        if len(found) > max_words:
            max_words = len(found)
        keys[candidate] = len(found)

print('Key \t\t Likelihood')
for likely_key in keys:
    print(f'{likely_key} \t\t {keys[likely_key]}')

Key 		 Likelihood
(1, 2) 		 4
(1, 6) 		 5
(1, 20) 		 6
(3, 5) 		 5
(3, 8) 		 6
(3, 10) 		 4
(3, 15) 		 4
(3, 24) 		 4
(5, 6) 		 7
(5, 18) 		 6
(5, 20) 		 4
(5, 22) 		 5
(5, 24) 		 7
(7, 0) 		 4
(7, 1) 		 4
(7, 4) 		 6
(7, 10) 		 4
(7, 11) 		 4
(7, 14) 		 4
(7, 20) 		 5
(7, 22) 		 5
(9, 0) 		 4
(9, 13) 		 5
(9, 15) 		 4
(9, 20) 		 4
(9, 22) 		 4
(11, 0) 		 5
(11, 8) 		 5
(11, 10) 		 5
(11, 11) 		 4
(11, 13) 		 5
(11, 22) 		 4
(11, 24) 		 5
(15, 7) 		 5
(15, 8) 		 7
(15, 10) 		 7
(15, 14) 		 4
(15, 21) 		 5
(15, 24) 		 5
(17, 6) 		 4
(17, 7) 		 4
(17, 25) 		 4
(19, 2) 		 4
(19, 7) 		 4
(19, 18) 		 8
(19, 20) 		 7
(21, 10) 		 5
(23, 0) 		 4
(23, 4) 		 5
(23, 8) 		 4
(23, 10) 		 5
(23, 11) 		 4
(23, 14) 		 6
(23, 16) 		 4
(25, 2) 		 8
(25, 12) 		 4
(25, 22) 		 4
(25, 24) 		 15
(25, 25) 		 4


Now that we have a list of keys and their likelihood of being correct, we can display the keys and the possible plain text, in-order of likelihood.

In [79]:
text_list = [[] for _ in range(max_words + 1)]
for likely_key in keys:
    count = keys[likely_key]
    text_list[count].append(affinedecipher(C, likely_key))

print('Key \t\t\t Likelihood')
for i in range(max_words, 2, -1):
    for text in text_list[i]:
        print(f'{text} \t\t\t {i}')

Key 			 Likelihood
ENEMYISGOINGTOATTACKTODAYPLEASEBEWARE 			 15
WBWMKEYAQEBANQONNOSINQHOKFXWOYWDWGOJW 			 8
IRIQCMWKSMRKXSEXXEGOXSHECTPIEWIFIAEVI 			 8
IBIWECASGCBSFGOFFOYMFGDOELRIOAITIUOVI 			 7
UNUIQOMESONERSARRAKYRSPAQXDUAMUFUGAHU 			 7
GVGCWEMSOEVSFOIFFIUQFONIWHJGIMGBGKITG 			 7
SHSOIQYEAQHERAURRUGCRAZUITVSUYSNSWUFS 			 7
AFAQOICEUIFERUSRRSWMRULSOJBASCAHAKSNA 			 7
ARASGWMYQWRYLQELLECULQBEGPTAEMADAIENA 			 6
EBEKGUIMSUBMZSOZZOWCZSNOGJTEOIEFEYORE 			 6
QJQEMKIAOKJANOWNNWGUNOLWMTZQWIQBQCWDQ 			 6
GBGQSYECMYBCPMOPPOKUPMVOSXFGOEGZGWOTG 			 6
YBYSWIUQKIBQDKODDOGADKPOWTJYOUYXYEOLY 			 6
OFOGUKAMEKFMZESZZSQIZEPSUDHOSAOROWSBO 			 5
FCFLHVJNTVCNATPAAPXDATOPHKUFPJFGFZPSF 			 5
KDKYGECUIEDUHIQHHQAOHIFQGNTKQCKVKWQXK 			 5
AVAKMSYWGSVWJGIJJIEOJGPIMRZAIYATAQINA 			 5
WRWGIOUSCORSFCEFFEAKFCLEINVWEUWPWMEJW 			 5
VUVXNJFPRJUPCRHCCHBDCRYHNOAVHFVEVTHIV 			 5
QBQUASKEISBERIORROCGRIJOAPNQOKQVQMODQ 			 5
UFUYEWOIMWFIVMSVVSGKVMNSETRUSOUZUQSHU 			 5
ITIMSKCWAKTWJAGJJGUYJABGSHFIGCINIEGVI 			 5
DODHNFXRVFOR